## Using LSTMs to Predict Mid-Price Movement from the Limit Order Book

- The goal of this notebook is to predict whether the mid-price will decrease, stay the same, or increase during the next tick. This is accomplished through the usage of an LSTM in tensorflow.

### Import the necessary packages

In [1]:
%matplotlib inline
from ml_analysis import MLOperator, MLEvaluator, get_LSTM_data
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import json
import time
import tensorflow as tf
from tensorflow.contrib import rnn
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc

/Users/meihuaren/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Prepare the data

In [2]:
df = pd.read_csv('all_features_new.csv')
df = df.dropna()
train_weight = 0.8
split = int(len(df)*train_weight)
df_train = df[:split]
df_test = df[split:]
operator = MLOperator()
n_steps = 5
use_features = list(df.columns[:-1])
y_train = df_train.label
sampled_idx = operator.get_samples_index(y_train.iloc[n_steps-1:], 'min')
x_train = df_train[use_features]
y_test = df_test.label.iloc[n_steps-1:]
x_test = df_test[use_features]
x_train, y_train, x_test, y_test = get_LSTM_data(x_train, y_train, x_test, y_test, sampled_idx, n_steps)
#normalization
x_max = np.max(x_train,axis=0)
x_min = np.min(x_train,axis=0)
x_train = (x_train - x_min) / (x_max - x_min)
x_test = (x_test - x_min) / (x_max - x_min)
nrow = 3000
x_valid = x_test[0:nrow]
y_valid = y_test[0:nrow]
x_test = x_test[nrow:]
y_test = y_test[nrow:]
#print (x_train.shape,y_train.shape,x_test.shape,y_test.shape, x_valid.shape, y_valid.shape)

In [3]:
df.label.value_counts()

 0    520883
-1      1539
 1      1401
Name: label, dtype: int64

### Set the parameters

In [4]:
# Training Parameters
learning_rate = 0.0001
n_epoch = 5000
n_batch = 700
display_step = 100

# Network Parameters
num_input = x_train.shape[-1] # number of features
timesteps = n_steps # timesteps
num_hidden = 4 # hidden layer num of features
num_classes = 3 # up, down and no-movement

# tf Graph input
# the input variables are first define as placeholder
# a placeholder is a variable/data which will be assigned later
X = tf.placeholder("float", [None, timesteps, num_input]) #dim: batch_size, number of time steps, number of inputs
Y = tf.placeholder("float", [None, num_classes])#dim:batch_size, number of classes (10 here)

#initialize the weigths with a normal random law initialization
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes], seed = 4720))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes], seed = 4720))
}

activation_combs = [(i,j) for i in [tf.nn.tanh, tf.nn.sigmoid, tf.nn.relu, tf.nn.elu]\
                          for j in [tf.nn.tanh, tf.nn.sigmoid, tf.nn.relu, tf.nn.elu]]

### Define the RNN

In [5]:
def RNN(x, weights, biases, activations):
    # activations: tuple, element of activation_combs.
    
    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)
    
    with tf.variable_scope(str(activations[0]).split(' ')[1]+'_'+str(activations[1]).split(' ')[1]):

        # 1-layer LSTM with num_hidden units
        #rnn_cell = rnn.BasicLSTMCell(num_hidden,activation=tf.nn.sigmoid)
        #rnn_cell = rnn.BasicLSTMCell(num_hidden)

        # 2-layer LSTM, each layer has num_hidden units. And you can wrap more layers together by doing list comprehension.
        rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(num_hidden,activation=activations[0]),\
                                     rnn.BasicLSTMCell(num_hidden,activation=activations[1])])

        # Get rnn cell output
        outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

        # (1) Linear activation, using rnn inner loop last output
        return tf.matmul(outputs[-1], weights['out']) + biases['out']


### Define the loss and optimzation & Run the model & Evaluation & Prediction

In [6]:
index_list = [str(activations[0]).split(' ')[1]+'_'+str(activations[1]).split(' ')[1] for activations in activation_combs]
accuracies_train = pd.DataFrame(columns = [-1,0,1], index = index_list)
AUCs_train = pd.DataFrame(columns = [-1,0,1], index = index_list)
accuracies_test = pd.DataFrame(columns = [-1,0,1], index = index_list)
AUCs_test = pd.DataFrame(columns = [-1,0,1], index = index_list)

count = 0
for activations in activation_combs:
    #activations = activation_combs[0]
    count += 1
    print('Using activation combination',str(count),':',str(activations[0]).split(' ')[1],'+',str(activations[1]).split(' ')[1])

    ### Define the loss and optimzation
    logits = RNN(X, weights, biases, activations)
    prediction = tf.nn.softmax(logits)
    #define the loss 
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
    #tf.equal: Returns the truth value of (x == y) element-wise.
    #tf.cast: Casts a tensor to a new type. --- here it casts from boolean to float
    #tf.argmax:Returns the index with the largest value across axes of a tensor. --- here along the axis of the vector
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    init = tf.global_variables_initializer()


    ### Start training
    with tf.Session() as sess:

        start_time = time.time()

        #please, make sure you changed for your own path 
        #log_files_path = '/Users/macbook/Desktop/DLFall2018/codes/DL project/tensorflow-master/logs'
        log_files_path = '/Users/meihuaren/personal/DL_logs/activation_try/' + str(count) + '/'

        #save and restore variables to and from checkpoints.
        saver = tf.train.Saver()

        # Run the initializer
        sess.run(init)

        #will work with this later
        #saver.restore(sess, log_files_path+'multi_layer/model-checkpoint-66000')

        loss_trace = []

        num_examples = x_train.shape[0]

        # Training cycle
        for epoch in range(n_epoch):

            avg_cost = 0.
            total_batch = int(num_examples/n_batch)

            # Shuffle the data
            np.random.seed(4720)
            perm = np.arange(num_examples)
            np.random.shuffle(perm)
            x_train = x_train[perm]
            y_train = y_train[perm]

            # Loop over all batches
            for i in range(total_batch):

                minibatch_x, minibatch_y = x_train[i*n_batch:(i+1)*n_batch], y_train[i*n_batch:(i+1)*n_batch]

                #minibatch_x, minibatch_y = mnist.train.next_batch(batch_size)

                # Fit training using batch data
                #the training is done using the training dataset
                sess.run(train_op, feed_dict={X: minibatch_x, Y: minibatch_y})
                # Compute average loss
                avg_cost += sess.run(loss_op, feed_dict={X: minibatch_x, Y: minibatch_y})/total_batch

            # Display logs per epoch step
            if epoch % display_step == 0:

                #the accuracy is evaluated using the validation dataset
                train_acc = sess.run(accuracy, feed_dict={X: x_train, Y: y_train})
                val_cost = sess.run(loss_op, feed_dict={X: x_valid, Y: y_valid})
                acc = sess.run(accuracy, feed_dict={X: x_valid, Y: y_valid})
                loss_trace.append(1-acc)    
                print("Epoch:", '%04d' % (epoch+1), "train_loss :", "{:0.4f}".format(avg_cost), "train_acc :", "{:0.4f}".format(train_acc), \
                      "val_loss :", "{:0.4f}".format(val_cost), "val_acc :", "{:0.4f}".format(acc))    

        #save to use later
        saver.save(sess, log_files_path)

        print("Optimization Finished!")
        #accuracy evaluated with the whole test dataset
        acc = sess.run(accuracy, feed_dict={X: x_test, Y: y_test})
        print("Test Accuracy:", acc)

        elapsed_time = time.time() - start_time
        print('Execution time (seconds) was %0.3f' % elapsed_time)


    ### Evaluate the model
    # returns a compiled model
    # identical to the previous one
    with tf.Session() as sess:
        saver = tf.train.Saver()
        saver.restore(sess, log_files_path)

        pred = sess.run(prediction, feed_dict={X: x_train, Y: y_train})    
        pred_df_train = pd.DataFrame(pred, columns=['-1', '0', '1'])
        pred_df_train['predict'] = pred_df_train.idxmax(axis=1)
        pred_df_train['true'] = pd.DataFrame(y_train, columns=['-1', '0', '1']).idxmax(axis=1)

        pred = sess.run(prediction, feed_dict={X: x_test, Y: y_test})    
        pred_df_test = pd.DataFrame(pred, columns=['-1', '0', '1'])
        pred_df_test['predict'] = pred_df_test.idxmax(axis=1)
        pred_df_test['true'] = pd.DataFrame(y_test, columns=['-1', '0', '1']).idxmax(axis=1)  

    train_f1 = f1_score(pred_df_train.true, pred_df_train.predict, average=None)
    test_f1 = f1_score(pred_df_test.true, pred_df_test.predict, average=None)
    print (train_f1)
    print (test_f1)


    ### Results on training data
    evaluator = MLEvaluator()
    evaluator.set_pred_df(pred_df_train)

    # compute & store accuracy
    cf_mx_train = evaluator.generate_confusion_matrix()
    cf_mx_train_normalized = cf_mx_train.values.astype('float') / cf_mx_train.values.sum(axis=1)[:, np.newaxis]
    row_idx = str(activations[0]).split(' ')[1]+'_'+str(activations[1]).split(' ')[1]
    accuracies_train.loc[row_idx, -1] = cf_mx_train_normalized[0][0]
    accuracies_train.loc[row_idx, 0] = cf_mx_train_normalized[1][1]
    accuracies_train.loc[row_idx, 1] = cf_mx_train_normalized[2][2]
    #print(accuracies)

    # compute & store AUC
    for lab in evaluator.pred_df.true.unique():
        fpr, tpr, thresholds = roc_curve(evaluator.pred_df.true, evaluator.pred_df[str(lab)], pos_label=lab)
        roc_auc = auc(fpr, tpr)
        AUCs_train.loc[row_idx,int(lab)] = roc_auc
        #print(lab,roc_auc)
    #print(AUCs)

    ### Results on testing data
    evaluator = MLEvaluator()
    evaluator.set_pred_df(pred_df_test)

    # compute & store accuracy
    cf_mx_train = evaluator.generate_confusion_matrix()
    cf_mx_train_normalized = cf_mx_train.values.astype('float') / cf_mx_train.values.sum(axis=1)[:, np.newaxis]
    row_idx = str(activations[0]).split(' ')[1]+'_'+str(activations[1]).split(' ')[1]
    accuracies_test.loc[row_idx, -1] = cf_mx_train_normalized[0][0]
    accuracies_test.loc[row_idx, 0] = cf_mx_train_normalized[1][1]
    accuracies_test.loc[row_idx, 1] = cf_mx_train_normalized[2][2]
    #print(accuracies)

    # compute & store AUC
    for lab in evaluator.pred_df.true.unique():
        fpr, tpr, thresholds = roc_curve(evaluator.pred_df.true, evaluator.pred_df[str(lab)], pos_label=lab)
        roc_auc = auc(fpr, tpr)
        AUCs_test.loc[row_idx,int(lab)] = roc_auc
        #print(lab,roc_auc)
    #print(AUCs)

Using activation combination 1 : tanh + tanh
Epoch: 0001 train_loss : 1.2952 train_acc : 0.3333 val_loss : 0.4511 val_acc : 0.9970
Epoch: 0101 train_loss : 1.1030 train_acc : 0.3333 val_loss : 0.8248 val_acc : 0.9970
Epoch: 0201 train_loss : 1.0272 train_acc : 0.4837 val_loss : 0.6970 val_acc : 0.9810
Epoch: 0301 train_loss : 0.9179 train_acc : 0.5560 val_loss : 0.6818 val_acc : 0.9583
Epoch: 0401 train_loss : 0.8278 train_acc : 0.6578 val_loss : 0.5853 val_acc : 0.9037
Epoch: 0501 train_loss : 0.7348 train_acc : 0.7098 val_loss : 0.4539 val_acc : 0.8120
Epoch: 0601 train_loss : 0.6453 train_acc : 0.7470 val_loss : 0.3894 val_acc : 0.7983
Epoch: 0701 train_loss : 0.5922 train_acc : 0.7706 val_loss : 0.3643 val_acc : 0.8157
Epoch: 0801 train_loss : 0.5605 train_acc : 0.7862 val_loss : 0.3438 val_acc : 0.8333
Epoch: 0901 train_loss : 0.5406 train_acc : 0.7934 val_loss : 0.3328 val_acc : 0.8387
Epoch: 1001 train_loss : 0.5259 train_acc : 0.7965 val_loss : 0.3247 val_acc : 0.8480
Epoch: 11

Epoch: 4201 train_loss : 0.4170 train_acc : 0.8379 val_loss : 0.1972 val_acc : 0.9090
Epoch: 4301 train_loss : 0.4193 train_acc : 0.8376 val_loss : 0.1995 val_acc : 0.9080
Epoch: 4401 train_loss : 0.4113 train_acc : 0.8385 val_loss : 0.2012 val_acc : 0.9067
Epoch: 4501 train_loss : 0.4113 train_acc : 0.8382 val_loss : 0.1982 val_acc : 0.9077
Epoch: 4601 train_loss : 0.4118 train_acc : 0.8401 val_loss : 0.1959 val_acc : 0.9083
Epoch: 4701 train_loss : 0.4059 train_acc : 0.8413 val_loss : 0.1968 val_acc : 0.9080
Epoch: 4801 train_loss : 0.4081 train_acc : 0.8421 val_loss : 0.1950 val_acc : 0.9073
Epoch: 4901 train_loss : 0.4042 train_acc : 0.8438 val_loss : 0.1920 val_acc : 0.9083
Optimization Finished!
Test Accuracy: 0.8264856
Execution time (seconds) was 114.652
INFO:tensorflow:Restoring parameters from /Users/meihuaren/personal/DL_logs/activation_try/2/
[0.8409725  0.86425339 0.83151515]
[0.02869229 0.90494342 0.03848224]
Using activation combination 3 : tanh + relu
Epoch: 0001 train_

Epoch: 3101 train_loss : 0.4278 train_acc : 0.8299 val_loss : 0.2616 val_acc : 0.8663
Epoch: 3201 train_loss : 0.4243 train_acc : 0.8296 val_loss : 0.2592 val_acc : 0.8667
Epoch: 3301 train_loss : 0.4236 train_acc : 0.8282 val_loss : 0.2545 val_acc : 0.8680
Epoch: 3401 train_loss : 0.4230 train_acc : 0.8296 val_loss : 0.2473 val_acc : 0.8723
Epoch: 3501 train_loss : 0.4171 train_acc : 0.8301 val_loss : 0.2480 val_acc : 0.8723
Epoch: 3601 train_loss : 0.4153 train_acc : 0.8307 val_loss : 0.2390 val_acc : 0.8740
Epoch: 3701 train_loss : 0.4155 train_acc : 0.8318 val_loss : 0.2350 val_acc : 0.8753
Epoch: 3801 train_loss : 0.4149 train_acc : 0.8335 val_loss : 0.2311 val_acc : 0.8763
Epoch: 3901 train_loss : 0.4111 train_acc : 0.8340 val_loss : 0.2260 val_acc : 0.8793
Epoch: 4001 train_loss : 0.4101 train_acc : 0.8346 val_loss : 0.2246 val_acc : 0.8793
Epoch: 4101 train_loss : 0.4086 train_acc : 0.8354 val_loss : 0.2191 val_acc : 0.8813
Epoch: 4201 train_loss : 0.4041 train_acc : 0.8351 val

Epoch: 2001 train_loss : 0.6208 train_acc : 0.7801 val_loss : 0.4459 val_acc : 0.8373
Epoch: 2101 train_loss : 0.6039 train_acc : 0.7848 val_loss : 0.4316 val_acc : 0.8377
Epoch: 2201 train_loss : 0.5927 train_acc : 0.7887 val_loss : 0.4170 val_acc : 0.8377
Epoch: 2301 train_loss : 0.5785 train_acc : 0.7907 val_loss : 0.4045 val_acc : 0.8383
Epoch: 2401 train_loss : 0.5681 train_acc : 0.7934 val_loss : 0.3940 val_acc : 0.8377
Epoch: 2501 train_loss : 0.5576 train_acc : 0.7968 val_loss : 0.3836 val_acc : 0.8387
Epoch: 2601 train_loss : 0.5469 train_acc : 0.8004 val_loss : 0.3740 val_acc : 0.8383
Epoch: 2701 train_loss : 0.5415 train_acc : 0.8029 val_loss : 0.3645 val_acc : 0.8387
Epoch: 2801 train_loss : 0.5315 train_acc : 0.8046 val_loss : 0.3563 val_acc : 0.8393
Epoch: 2901 train_loss : 0.5255 train_acc : 0.8054 val_loss : 0.3475 val_acc : 0.8410
Epoch: 3001 train_loss : 0.5180 train_acc : 0.8101 val_loss : 0.3414 val_acc : 0.8417
Epoch: 3101 train_loss : 0.5119 train_acc : 0.8112 val

Epoch: 0901 train_loss : 0.7104 train_acc : 0.7412 val_loss : 0.4993 val_acc : 0.8457
Epoch: 1001 train_loss : 0.6845 train_acc : 0.7548 val_loss : 0.4437 val_acc : 0.8517
Epoch: 1101 train_loss : 0.6588 train_acc : 0.7637 val_loss : 0.3966 val_acc : 0.8590
Epoch: 1201 train_loss : 0.6383 train_acc : 0.7701 val_loss : 0.3576 val_acc : 0.8633
Epoch: 1301 train_loss : 0.6205 train_acc : 0.7734 val_loss : 0.3329 val_acc : 0.8650
Epoch: 1401 train_loss : 0.6024 train_acc : 0.7770 val_loss : 0.3107 val_acc : 0.8653
Epoch: 1501 train_loss : 0.5881 train_acc : 0.7795 val_loss : 0.2904 val_acc : 0.8720
Epoch: 1601 train_loss : 0.5800 train_acc : 0.7854 val_loss : 0.2812 val_acc : 0.8720
Epoch: 1701 train_loss : 0.5731 train_acc : 0.7890 val_loss : 0.2667 val_acc : 0.8760
Epoch: 1801 train_loss : 0.5599 train_acc : 0.7890 val_loss : 0.2556 val_acc : 0.8777
Epoch: 1901 train_loss : 0.5555 train_acc : 0.7909 val_loss : 0.2499 val_acc : 0.8803
Epoch: 2001 train_loss : 0.5451 train_acc : 0.7932 val

[0.84473993 0.86459286 0.83068135]
[0.02767482 0.88101637 0.02905379]
Using activation combination 10 : relu + sigmoid
Epoch: 0001 train_loss : 1.1054 train_acc : 0.3333 val_loss : 0.9271 val_acc : 0.9970
Epoch: 0101 train_loss : 0.9749 train_acc : 0.5835 val_loss : 1.0389 val_acc : 0.3523
Epoch: 0201 train_loss : 0.8205 train_acc : 0.6931 val_loss : 0.7093 val_acc : 0.8053
Epoch: 0301 train_loss : 0.7159 train_acc : 0.7470 val_loss : 0.5091 val_acc : 0.8337
Epoch: 0401 train_loss : 0.6478 train_acc : 0.7729 val_loss : 0.4588 val_acc : 0.8197
Epoch: 0501 train_loss : 0.6104 train_acc : 0.7848 val_loss : 0.4275 val_acc : 0.8290
Epoch: 0601 train_loss : 0.5847 train_acc : 0.7940 val_loss : 0.3949 val_acc : 0.8493
Epoch: 0701 train_loss : 0.5635 train_acc : 0.8009 val_loss : 0.3633 val_acc : 0.8670
Epoch: 0801 train_loss : 0.5474 train_acc : 0.8046 val_loss : 0.3488 val_acc : 0.8750
Epoch: 0901 train_loss : 0.5347 train_acc : 0.8043 val_loss : 0.3214 val_acc : 0.8857
Epoch: 1001 train_los

Epoch: 4101 train_loss : 0.6725 train_acc : 0.6636 val_loss : 0.9066 val_acc : 0.8320
Epoch: 4201 train_loss : 0.6720 train_acc : 0.6631 val_loss : 0.9032 val_acc : 0.8360
Epoch: 4301 train_loss : 0.6703 train_acc : 0.6675 val_loss : 0.9025 val_acc : 0.8377
Epoch: 4401 train_loss : 0.6667 train_acc : 0.6633 val_loss : 0.8929 val_acc : 0.8423
Epoch: 4501 train_loss : 0.6678 train_acc : 0.6686 val_loss : 0.8840 val_acc : 0.8450
Epoch: 4601 train_loss : 0.6640 train_acc : 0.6717 val_loss : 0.8993 val_acc : 0.8433
Epoch: 4701 train_loss : 0.6630 train_acc : 0.6731 val_loss : 0.8854 val_acc : 0.8457
Epoch: 4801 train_loss : 0.6621 train_acc : 0.6750 val_loss : 0.8880 val_acc : 0.8457
Epoch: 4901 train_loss : 0.6604 train_acc : 0.6772 val_loss : 0.8971 val_acc : 0.8450
Optimization Finished!
Test Accuracy: 0.804886
Execution time (seconds) was 118.703
INFO:tensorflow:Restoring parameters from /Users/meihuaren/personal/DL_logs/activation_try/11/
[0.86520875 0.67257893 0.42795699]
[0.0256917  

Epoch: 3001 train_loss : 0.4336 train_acc : 0.8276 val_loss : 0.2185 val_acc : 0.9147
Epoch: 3101 train_loss : 0.4322 train_acc : 0.8274 val_loss : 0.2224 val_acc : 0.9140
Epoch: 3201 train_loss : 0.4302 train_acc : 0.8276 val_loss : 0.2153 val_acc : 0.9157
Epoch: 3301 train_loss : 0.4245 train_acc : 0.8287 val_loss : 0.2216 val_acc : 0.9143
Epoch: 3401 train_loss : 0.4257 train_acc : 0.8299 val_loss : 0.2193 val_acc : 0.9157
Epoch: 3501 train_loss : 0.4233 train_acc : 0.8307 val_loss : 0.2203 val_acc : 0.9150
Epoch: 3601 train_loss : 0.4170 train_acc : 0.8324 val_loss : 0.2223 val_acc : 0.9147
Epoch: 3701 train_loss : 0.4176 train_acc : 0.8321 val_loss : 0.2230 val_acc : 0.9150
Epoch: 3801 train_loss : 0.4155 train_acc : 0.8332 val_loss : 0.2257 val_acc : 0.9140
Epoch: 3901 train_loss : 0.4117 train_acc : 0.8338 val_loss : 0.2298 val_acc : 0.9100
Epoch: 4001 train_loss : 0.4098 train_acc : 0.8346 val_loss : 0.2312 val_acc : 0.9060
Epoch: 4101 train_loss : 0.4106 train_acc : 0.8343 val

Epoch: 1901 train_loss : 0.4915 train_acc : 0.8126 val_loss : 0.3972 val_acc : 0.8757
Epoch: 2001 train_loss : 0.4815 train_acc : 0.8185 val_loss : 0.3868 val_acc : 0.8687
Epoch: 2101 train_loss : 0.4786 train_acc : 0.8204 val_loss : 0.3761 val_acc : 0.8683
Epoch: 2201 train_loss : 0.4720 train_acc : 0.8251 val_loss : 0.3629 val_acc : 0.8683
Epoch: 2301 train_loss : 0.4637 train_acc : 0.8279 val_loss : 0.3506 val_acc : 0.8697
Epoch: 2401 train_loss : 0.4579 train_acc : 0.8282 val_loss : 0.3448 val_acc : 0.8700
Epoch: 2501 train_loss : 0.4530 train_acc : 0.8304 val_loss : 0.3346 val_acc : 0.8720
Epoch: 2601 train_loss : 0.4478 train_acc : 0.8293 val_loss : 0.3295 val_acc : 0.8717
Epoch: 2701 train_loss : 0.4445 train_acc : 0.8324 val_loss : 0.3208 val_acc : 0.8727
Epoch: 2801 train_loss : 0.4424 train_acc : 0.8326 val_loss : 0.3101 val_acc : 0.8753
Epoch: 2901 train_loss : 0.4368 train_acc : 0.8340 val_loss : 0.3076 val_acc : 0.8750
Epoch: 3001 train_loss : 0.4351 train_acc : 0.8332 val

### Output accuracy & AUC

In [8]:
# ouput
accuracies_train.to_csv('accuracies_train.csv')
AUCs_train.to_csv('AUCs_train.csv')
accuracies_test.to_csv('accuracies_test.csv')
AUCs_test.to_csv('AUCs_test.csv')